In [ ]:
%reset
%load_ext autoreload
%autoreload 2

import Sampling
import UsefulFunctions as fun
import DeformationModules as df
import Models
import Hamiltonian

import numpy as np
import matplotlib.pyplot as plt
import torch

torch.set_default_tensor_type(torch.DoubleTensor)

In [ ]:
source = Sampling.sampleFromGreyscale("data/density_a.png", threshold=0.5, centered=True)
target = Sampling.sampleFromGreyscale("data/density_b.png", threshold=0.5, centered=True)

In [ ]:
minx, miny, maxx, maxy = torch.min(source[0][:, 0]).detach().numpy(), torch.min(source[0][:, 1]).detach().numpy(), torch.max(source[0][:, 0]).detach().numpy(), torch.max(source[0][:, 1]).detach().numpy()

In [ ]:
sigma = 0.05
x, y = torch.meshgrid([torch.arange(minx-sigma, maxx+sigma, step=sigma), torch.arange(miny-sigma, maxy+sigma, step=sigma)])
GD = fun.grid2vec(x, y)
GD = GD.contiguous().view(-1)

In [ ]:
fun.plotTensorScatter(source, alpha=0.4)
fun.plotTensorScatter(target, alpha=0.4)
plt.plot(GD.view(-1, 2).detach()[:, 1].numpy(), GD.view(-1, 2).detach()[:, 0].numpy(), '.')
plt.show()

In [ ]:
myModel = Models.ModelTranslationModuleRegistration(sigma, 2, source, GD, True)
costs = myModel.fit(target, maxiter=10, l=100., lr=1e-4, logInterval=5)

In [ ]:
out = myModel()
outGD, _ = myModel.shootList()
%matplotlib qt5
plt.subplot(1, 2, 1)
fun.plotTensorScatter(target, alpha=0.4)
fun.plotTensorScatter(out, alpha=0.4)
plt.plot(outGD[1].view(-1, 2).detach()[:, 1].numpy(), outGD[1].view(-1, 2).detach()[:, 0].numpy(), '.')
plt.subplot(1, 2, 2)
fun.plotTensorScatter(target, alpha=0.4)
fun.plotTensorScatter(source, alpha=0.4)
plt.show()


In [ ]:
plt.plot(range(0, len(costs)), costs)
plt.show()